In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [4]:
import PIL.Image as Image
import cv2
import os

**Downloading the already trained model from tensorflow_hub**

In [5]:
classifier = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4",input_shape=(224,224,3))
])

In [6]:
data_source = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'
flowers_data = tf.keras.utils.get_file('flower_photos',origin=data_source,cache_dir='.',untar=True)

In [7]:
import pathlib

In [8]:
flowers_data = pathlib.Path(flowers_data)

In [9]:
len(list(flowers_data.glob('*/*.jpg')))

3670

In [9]:
flower_images_dict = {
    'dandelion': list(flowers_data.glob('dandelion/*')),
    'roses': list(flowers_data.glob('roses/*')),
    'tulips': list(flowers_data.glob('tulips/*')),
    'sunflowers': list(flowers_data.glob('sunflowers/*')),
    'daisy': list(flowers_data.glob('daisy/*')),
}

In [10]:
flower_dict_labels = {
    'dandelion':0,
    'roses':1,
    'tulips':2,
    'sunflowers':3,
    'daisy':4
}

In [11]:
x,y = [],[]

for flower_name,images in flower_images_dict.items():
  for image in images:
    img = cv2.imread(str(image))
    resized_image = cv2.resize(img,(224,224))
    x.append(resized_image)
    y.append(flower_dict_labels[flower_name])

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
x = np.array(x)
y = np.array(y) 

In [14]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [15]:
x_train_scaled = np.divide(x_train,255)
x_test_scaled = np.divide(x_test,255)

**We create download a model without the last layer i.e the feature vectors**

In [16]:
feature_vector = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",input_shape=(224,224,3),trainable=False)

In [17]:
model = tf.keras.Sequential([
    feature_vector,
    tf.keras.layers.Dense(5)
])


In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [18]:
model.compile(
    optimizer='adam',
    metrics =['accuracy'],
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

In [19]:
model.fit(x_train_scaled,y_train,epochs=5)

Epoch 1/5
92/92 [==============================] - 17s 46ms/step - loss: 0.7421 - accuracy: 0.7340
Epoch 2/5
92/92 [==============================] - 4s 45ms/step - loss: 0.3822 - accuracy: 0.8747
Epoch 3/5
92/92 [==============================] - 4s 43ms/step - loss: 0.3026 - accuracy: 0.9040
Epoch 4/5
92/92 [==============================] - 4s 42ms/step - loss: 0.2494 - accuracy: 0.9281
Epoch 5/5
92/92 [==============================] - 4s 43ms/step - loss: 0.2179 - accuracy: 0.9394
